In [1]:
import csv
import math
import os

import numpy as np

from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.pipeline import *
from sklearn.tree import *


from sklearn.cross_validation import *
from sklearn.decomposition import *
from sklearn.ensemble import *
from sklearn.feature_selection import *
from sklearn.grid_search import *

In [2]:
import pandas

In [19]:
%%bash
if [[ ! -f 'atlas-higgs-challenge-2014-v2.csv' ]]; then \
    wget -c http://opendata.cern.ch/record/328/files/atlas-higgs-challenge-2014-v2.csv.gz ; \
    gunzip 'atlas-higgs-challenge-2014-v2.csv.gz' ; \
fi

In [3]:
pd = pandas.read_csv('atlas-higgs-challenge-2014-v2.csv')

In [4]:
pd['Label'] = pd['Label'] == 's'

In [5]:
pd_train = pd[pd['KaggleSet'] == 't']

In [7]:
X = pd_train.iloc[:,1:31]

In [8]:
Y = pd_train.iloc[:,32]

In [9]:
W = pd_train['KaggleWeight']

In [10]:
X

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,138.470,51.655,97.827,27.980,0.910,124.711,2.666,3.064,41.928,197.760,...,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.240,-2.475,113.497
1,160.937,68.768,103.235,48.146,-999.000,-999.000,-999.000,3.473,2.078,125.157,...,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.000,-999.000,46.226
2,-999.000,162.172,125.953,35.635,-999.000,-999.000,-999.000,3.148,9.336,197.814,...,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.000,-999.000,44.251
3,143.905,81.417,80.943,0.414,-999.000,-999.000,-999.000,3.310,0.414,75.968,...,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-0.000
4,175.864,16.915,134.805,16.405,-999.000,-999.000,-999.000,3.891,16.405,57.983,...,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
5,89.744,13.550,59.149,116.344,2.636,284.584,-0.540,1.362,61.619,278.876,...,2.237,282.849,3,90.547,-2.412,-0.653,56.165,0.224,3.106,193.660
6,148.754,28.862,107.782,106.130,0.733,158.359,0.113,2.941,2.545,305.967,...,-1.443,294.074,2,123.010,0.864,1.450,56.867,0.131,-2.767,179.877
7,154.916,10.418,94.714,29.169,-999.000,-999.000,-999.000,2.897,1.526,138.178,...,-1.761,187.299,1,30.638,-0.715,-1.724,-999.000,-999.000,-999.000,30.638
8,105.594,50.559,100.989,4.288,-999.000,-999.000,-999.000,2.904,4.288,65.333,...,0.024,129.804,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
9,128.053,88.941,69.272,193.392,-999.000,-999.000,-999.000,1.609,28.859,255.123,...,0.845,294.741,1,167.735,-2.767,-2.514,-999.000,-999.000,-999.000,167.735


In [20]:
pd_test = pd[pd['KaggleSet'] == 'p']

In [21]:
idstest = pd_test['EventId']
Xts = pd_test.iloc[:,1:31]
Wts = pd_test['KaggleWeight']

skip missing values and convert positive variables into logarithmic scale

In [ ]:
imputer = Imputer(missing_values = -999.0, strategy = 'most_frequent')
X = imputer.fit_transform(X)
Xts = imputer.transform(Xts)


In [ ]:
indTransform = (0,1,2,3,4,5,7,8,9,10,12,13,16,19,21,23,26)
Xtrans = np.log(1 / (1 + X[:, indTransform]))
X = np.hstack((X, Xtrans))

XtransTest = np.log(1 / (1 + Xts[:, indTransform]))
Xts = np.hstack((Xts, XtransTest))


### make variances equal


In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
Xts = scaler.transform(Xts)
print "model init"


### Adaboost + trees?

1. trees are easy to boost
2. adaboost is not that bad
3. trees are bagging-friendly. 
4. parameters are taken from competition forum.


In [ ]:
rf = ExtraTreesClassifier(
            n_estimators = 300,
            max_features = 30,
            max_depth = 12,
            min_samples_leaf = 100,
            min_samples_split = 100,
            verbose = 1,
            n_jobs = -1)
classifier = AdaBoostClassifier(
        n_estimators = 20,
        learning_rate = 0.75,
        base_estimator = rf)
print "model fit"
classifier.fit(X, Y, sample_weight = W)


In [ ]:
print "predict"
Ypred = classifier.predict_proba(X)[:,1]
Yts_pred = classifier.predict_proba(Xts)[:,1]

#last:15
signal_threshold = 84
cut = np.percentile(Yts_pred, signal_threshold)
Ysig = Ypred > cut
Yts_sig = Yts_pred > cut

print "Saving results"
#пишем csv
idsProbs = np.transpose(np.vstack((idstest, Yts_pred)))
idsProbs = np.array(sorted(idsProbs, key = lambda x: -x[1]))
idsRanks = np.hstack((
    idsProbs,
    np.arange(1, idsProbs.shape[0]+1).reshape((idsProbs.shape[0], 1))))

idsMap = {}
for tsid, prob, rank in idsRanks:
    tsid = int(tsid)
    rank = int(rank)
    idsMap[tsid] = rank

f = open('higgs.pred.csv', 'wb')
writer = csv.writer(f)
writer.writerow(['EventId', 'RankOrder', 'Class'])
for i, pred in enumerate(Yts_sig):
    event_id = int(idstest[i])
    rank = idsMap[idstest[i]]
    klass = pred and 's' or 'b'
    writer.writerow([event_id, rank, klass])
f.close()



def ams(s, b):
    return math.sqrt(2 * ((s + b + 10) * math.log(1.0 + s/(b + 10)) - s))

